## Preprocessing

In [194]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [195]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME' and 'STATUS'
application_df.drop(columns=['EIN','NAME', 'STATUS'], inplace=True)

In [196]:
# Create bins for the ASK_AMT values.
application_df = application_df.join(pd.cut(application_df.pop('ASK_AMT'),
                     bins=[0,5000,1000000,9000000000],
                     labels=['5000 or less', '5000 to 1mil', 'more than 1mil'],
                     include_lowest=True,).to_frame('ASK_AMT_BINS'))

In [197]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
IS_SUCCESSFUL              2
ASK_AMT_BINS               3
dtype: int64

In [198]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [199]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types = pd.DataFrame(application_df['APPLICATION_TYPE'].value_counts()).reset_index()

application_types_to_replace = []

for index, row in application_types.iterrows():
  if row['APPLICATION_TYPE'] < 100:
    application_types_to_replace.append(row['index'])
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [200]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [201]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [202]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications = pd.DataFrame(application_df['CLASSIFICATION'].value_counts()).reset_index()

classifications_to_replace = []

for index, row in classifications.iterrows():
  if row['CLASSIFICATION'] < 100:
    classifications_to_replace.append(row['index'])
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [203]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [204]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [205]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [206]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
from tensorflow import keras
from tensorflow.keras import layers

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(layers.Dense(units=500, activation='relu', input_dim=51))

# Additional hidden layers
nn.add(layers.Dense(units=100, activation='elu'))
nn.add(layers.Dropout(rate=.5))
nn.add(layers.Dense(units=200, activation='sigmoid'))
nn.add(layers.Dropout(rate=.5))
nn.add(layers.Dense(units=400, activation='tanh'))

# Output layer
nn.add(layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 500)               26000     
                                                                 
 dense_59 (Dense)            (None, 100)               50100     
                                                                 
 dropout_16 (Dropout)        (None, 100)               0         
                                                                 
 dense_60 (Dense)            (None, 200)               20200     
                                                                 
 dropout_17 (Dropout)        (None, 200)               0         
                                                                 
 dense_61 (Dense)            (None, 400)               80400     
                                                                 
 dense_62 (Dense)            (None, 1)               

In [207]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [208]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 7s 6ms/step - loss: 0.5958 - accuracy: 0.7004
Epoch 2/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5667 - accuracy: 0.7242
Epoch 3/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5613 - accuracy: 0.7250
Epoch 4/100
804/804 [==============================] - 4s 6ms/step - loss: 0.5584 - accuracy: 0.7289
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5573 - accuracy: 0.7267
Epoch 6/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5543 - accuracy: 0.7304
Epoch 7/100
804/804 [==============================] - 5s 7ms/step - loss: 0.5548 - accuracy: 0.7287
Epoch 8/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5527 - accuracy: 0.7281
Epoch 9/100
804/804 [==============================] - 5s 7ms/step - loss: 0.5515 - accuracy: 0.7310
Epoch 10/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5512 - accura

In [209]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5799 - accuracy: 0.7198 - 966ms/epoch - 4ms/step
Loss: 0.5799232125282288, Accuracy: 0.7197667360305786


In [210]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.keras')